Study these topics: bias vs. variance tradeoff, bagging and boosting approaches, ensemble approach, model complexity, and XGBoost algorithm.

Apply XGBoost classification algorithm on the loan data set and compare its performance against SVM and Logistic Regression by suitably splitting the data into train and test sets.

Apply XGBoost regression algorithm on the Boston housing data set and compare its performance against Linear regression.

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.tree import DecisionTreeClassifier

from sklearn.tree import plot_tree
import seaborn as sns 
from sklearn import model_selection

In [4]:
import os

os.getcwd()

'/Users/shubhamgoyal/Downloads/DataSciece19098'

In [5]:
loan_df= pd.read_excel('loan.xlsx')
loan_df.head()


,Sex,Age,Time_at_address,Res_status,Telephone,Occupation,Job_status,Time_employed,Time_bank,Liab_ref,Acc_ref,Home_Expn,Balance,Decision
0,M,50.750000,0.585,owner,given,unemploye,unemploye,0,0,f,given,145,0,reject
1,M,19.670000,10.000,rent,not_given,labourer,governmen,0,0,t,given,140,0,reject
2,F,52.830002,15.000,owner,given,creative_,private_s,5,14,f,given,0,2200,accept
3,M,22.670000,2.540,rent,not_given,creative_,governmen,2,0,f,given,0,0,accept
4,M,29.250000,13.000,owner,given,driver,governmen,0,0,f,given,228,0,reject


In [101]:
#one hot encoding product_info_2 var as it is string categorical variable

from sklearn.preprocessing import OneHotEncoder

OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
col_enc = ['Res_status', 'Liab_ref', 'Job_status', 'Acc_ref', 'Telephone', 'Sex', 'Occupation']

# Use the one-hot encoder to transform the categorical data columns. 
loan_df_enc = pd.DataFrame(OH_encoder.fit_transform(loan_df[col_enc]))

#reassigning the original index.
loan_df_enc.index = loan_df_enc.index

# Addings columns as per the encoding 
loan_df_enc.columns = OH_encoder.get_feature_names(col_enc)

# Create dataframes that only include the numerical features/columns (these will be concatenated with the one-hot encoded dataframes).
loan_df_no_enc = loan_df.drop(col_enc, axis=1)

# Concatenate the one-hot encoded columns with the existing numerical features/columns.
loan_df_upd = pd.concat([loan_df_no_enc, loan_df_enc], axis=1)
loan_df_upd.head()

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Age,Time_at_address,Time_employed,Time_bank,Home_Expn,Balance,Decision,Res_status_owner,Res_status_rent,Liab_ref_f,...,Occupation_executive,Occupation_guard_etc,Occupation_labourer,Occupation_manager,Occupation_office_st,Occupation_productio,Occupation_professio,Occupation_sales,Occupation_semi_pro,Occupation_unemploye
0,50.750000,0.585,0,0,145,0,reject,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,19.670000,10.000,0,0,140,0,reject,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,52.830002,15.000,5,14,0,2200,accept,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,22.670000,2.540,2,0,0,0,accept,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,29.250000,13.000,0,0,228,0,reject,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [102]:
from sklearn.preprocessing import LabelEncoder

labelencoder = LabelEncoder()

loan_df_upd['Decision_cat'] = labelencoder.fit_transform(loan_df_upd['Decision'])

loan_df_upd.drop('Decision',inplace=True,axis=1)

loan_df_upd.head()
#label encoding the dependent variable (Decision) to convert it into numerical

,Age,Time_at_address,Time_employed,Time_bank,Home_Expn,Balance,Res_status_owner,Res_status_rent,Liab_ref_f,Liab_ref_t,...,Occupation_guard_etc,Occupation_labourer,Occupation_manager,Occupation_office_st,Occupation_productio,Occupation_professio,Occupation_sales,Occupation_semi_pro,Occupation_unemploye,Decision_cat
0,50.750000,0.585,0,0,145,0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
1,19.670000,10.000,0,0,140,0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,52.830002,15.000,5,14,0,2200,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,22.670000,2.540,2,0,0,0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,29.250000,13.000,0,0,228,0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [103]:
# Feature scaling to normalise the data
loan_df_scaled=loan_df_upd.copy()

from sklearn.preprocessing import normalize
scale_cols=['Age','Time_at_address','Time_employed','Time_bank','Home_Expn','Balance']

loan_df_scaled[scale_cols]=normalize(loan_df_scaled[scale_cols])
loan_df_scaled.head()


,Age,Time_at_address,Time_employed,Time_bank,Home_Expn,Balance,Res_status_owner,Res_status_rent,Liab_ref_f,Liab_ref_t,...,Occupation_guard_etc,Occupation_labourer,Occupation_manager,Occupation_office_st,Occupation_productio,Occupation_professio,Occupation_sales,Occupation_semi_pro,Occupation_unemploye,Decision_cat
0,0.330348,0.003808,0.000000,0.000000,0.943852,0.000000,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
1,0.138787,0.070558,0.000000,0.000000,0.987806,0.000000,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,0.024006,0.006816,0.002272,0.006362,0.000000,0.999666,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.989984,0.110920,0.087339,0.000000,0.000000,0.000000,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.127044,0.056464,0.000000,0.000000,0.990289,0.000000,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [155]:
#splitting data into train-test groups

X=loan_df_scaled.drop('Decision_cat',axis=1)
y=loan_df_scaled['Decision_cat']

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2, random_state= 1)

In [156]:
print(X.shape,X_train.shape,X_test.shape)

(429, 35) (343, 35) (86, 35)


In [157]:
# Decision Tree - Using gridserch Cross validation for optimal training through a grid of parameters

dt = DecisionTreeClassifier(random_state=42)

# criterion=['gini','entropy']÷÷

# declare parameters for hyperparameter tuning
parameters = {'criterion':['gini','entropy']}




grid_search_dt = GridSearchCV(estimator = dt,  
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv=5,
                           verbose=3)


grid_search_dt.fit(X_train, y_train)

Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV 1/5] END ....................criterion=gini;, score=0.667 total time=   0.0s
[CV 2/5] END ....................criterion=gini;, score=0.652 total time=   0.0s
[CV 3/5] END ....................criterion=gini;, score=0.739 total time=   0.0s
[CV 4/5] END ....................criterion=gini;, score=0.765 total time=   0.0s
[CV 5/5] END ....................criterion=gini;, score=0.691 total time=   0.0s
[CV 1/5] END .................criterion=entropy;, score=0.739 total time=   0.0s
[CV 2/5] END .................criterion=entropy;, score=0.623 total time=   0.0s
[CV 3/5] END .................criterion=entropy;, score=0.725 total time=   0.0s
[CV 4/5] END .................criterion=entropy;, score=0.765 total time=   0.0s
[CV 5/5] END .................criterion=entropy;, score=0.588 total time=   0.0s


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'criterion': ['gini', 'entropy']}, scoring='accuracy',
             verbose=3)

In [158]:
# examine the best model


# best score achieved during the GridSearchCV (Takes the mean of all the CV folds) 
print('GridSearch CV best score : {:.4f}\n\n'.format(grid_search_dt.best_score_))


# print parameters that give the best results
print('Parameters that give the best results :','\n\n', (grid_search_dt.best_params_))


# print estimator that was chosen by the GridSearch
print('\n\nEstimator that was chosen by the search :','\n\n', (grid_search_dt.best_estimator_))

GridSearch CV best score : 0.7028


Parameters that give the best results : 

 {'criterion': 'gini'}


Estimator that was chosen by the search : 

 DecisionTreeClassifier(random_state=42)


In [159]:
y_pred_dt=grid_search_dt.predict(X_test)

from sklearn.metrics import classification_report

print(classification_report(y_test,y_pred_dt)) 

train_acc_dt= grid_search_dt.best_score_
acc_dt=classification_report(y_test,y_pred_dt,output_dict=True)['accuracy']


              precision    recall  f1-score   support

           0       0.79      0.58      0.67        45
           1       0.64      0.83      0.72        41

    accuracy                           0.70        86
   macro avg       0.71      0.70      0.70        86
weighted avg       0.72      0.70      0.69        86



In [160]:
hd = list(X_train.columns)
for i, f in zip(hd, grid_search_dt.best_estimator_.feature_importances_):
     print(i,round(f*100,2))

Age 10.34
Time_at_address 4.72
Time_employed 16.78
Time_bank 27.23
Home_Expn 10.94
Balance 9.84
Res_status_owner 0.0
Res_status_rent 0.0
Liab_ref_f 1.66
Liab_ref_t 0.0
Job_status_governmen 0.0
Job_status_military 0.0
Job_status_private_s 0.0
Job_status_retired 0.0
Job_status_self_empl 0.0
Job_status_student 0.0
Job_status_unemploye 0.0
Acc_ref_given 0.0
Acc_ref_oth_inst_ 1.69
Telephone_given 1.08
Telephone_not_given 0.79
Sex_F 0.0
Sex_M 1.3
Occupation_creative_ 2.54
Occupation_driver 0.0
Occupation_executive 0.0
Occupation_guard_etc 0.0
Occupation_labourer 0.0
Occupation_manager 1.99
Occupation_office_st 2.79
Occupation_productio 0.4
Occupation_professio 1.95
Occupation_sales 1.0
Occupation_semi_pro 1.5
Occupation_unemploye 1.46


In [109]:
#SVM- Using gridserch Cross validation for optimal training through a grid of parameters

from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

svc=SVC() 

# declare parameters for hyperparameter tuning
svc_parameters = [ {'C':[0.1,1, 10], 'kernel':['linear']},
               {'C':[0.1,1, 10], 'kernel':['rbf'], 'gamma':[0,0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]},
               {'C':[0.1,1, 10], 'kernel':['poly'], 'degree': [0,2,3,4] ,'gamma':[0,0.01,0.02,0.03,0.04,0.05]},
               {'C':[0.1,1, 10], 'kernel':['sigmoid']}
              
              ]

svc_grid_search = GridSearchCV(estimator = svc,  
                           param_grid = svc_parameters,
                           scoring = 'accuracy',
                           cv=5,
                           verbose=3)


svc_grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
[CV 1/5] END ..............C=0.1, kernel=linear;, score=0.623 total time=   0.0s
[CV 2/5] END ..............C=0.1, kernel=linear;, score=0.652 total time=   0.0s
[CV 3/5] END ..............C=0.1, kernel=linear;, score=0.710 total time=   0.0s
[CV 4/5] END ..............C=0.1, kernel=linear;, score=0.765 total time=   0.0s
[CV 5/5] END ..............C=0.1, kernel=linear;, score=0.735 total time=   0.0s
[CV 1/5] END ................C=1, kernel=linear;, score=0.652 total time=   0.0s
[CV 2/5] END ................C=1, kernel=linear;, score=0.696 total time=   0.0s
[CV 3/5] END ................C=1, kernel=linear;, score=0.739 total time=   0.0s
[CV 4/5] END ................C=1, kernel=linear;, score=0.750 total time=   0.0s
[CV 5/5] END ................C=1, kernel=linear;, score=0.735 total time=   0.0s
[CV 1/5] END ...............C=10, kernel=linear;, score=0.667 total time=   0.0s
[CV 2/5] END ...............C=10, kernel=linea

[CV 2/5] END ........C=1, gamma=0.9, kernel=rbf;, score=0.594 total time=   0.0s
[CV 3/5] END ........C=1, gamma=0.9, kernel=rbf;, score=0.623 total time=   0.0s
[CV 4/5] END ........C=1, gamma=0.9, kernel=rbf;, score=0.603 total time=   0.0s
[CV 5/5] END ........C=1, gamma=0.9, kernel=rbf;, score=0.603 total time=   0.0s
[CV 1/5] END .........C=10, gamma=0, kernel=rbf;, score=0.565 total time=   0.0s
[CV 2/5] END .........C=10, gamma=0, kernel=rbf;, score=0.565 total time=   0.0s
[CV 3/5] END .........C=10, gamma=0, kernel=rbf;, score=0.580 total time=   0.0s
[CV 4/5] END .........C=10, gamma=0, kernel=rbf;, score=0.574 total time=   0.0s
[CV 5/5] END .........C=10, gamma=0, kernel=rbf;, score=0.574 total time=   0.0s
[CV 1/5] END .......C=10, gamma=0.1, kernel=rbf;, score=0.667 total time=   0.0s
[CV 2/5] END .......C=10, gamma=0.1, kernel=rbf;, score=0.638 total time=   0.0s
[CV 3/5] END .......C=10, gamma=0.1, kernel=rbf;, score=0.667 total time=   0.0s
[CV 4/5] END .......C=10, ga

[CV 2/5] END C=0.1, degree=2, gamma=0.04, kernel=poly;, score=0.565 total time=   0.0s
[CV 3/5] END C=0.1, degree=2, gamma=0.04, kernel=poly;, score=0.580 total time=   0.0s
[CV 4/5] END C=0.1, degree=2, gamma=0.04, kernel=poly;, score=0.574 total time=   0.0s
[CV 5/5] END C=0.1, degree=2, gamma=0.04, kernel=poly;, score=0.574 total time=   0.0s
[CV 1/5] END C=0.1, degree=2, gamma=0.05, kernel=poly;, score=0.565 total time=   0.0s
[CV 2/5] END C=0.1, degree=2, gamma=0.05, kernel=poly;, score=0.565 total time=   0.0s
[CV 3/5] END C=0.1, degree=2, gamma=0.05, kernel=poly;, score=0.580 total time=   0.0s
[CV 4/5] END C=0.1, degree=2, gamma=0.05, kernel=poly;, score=0.574 total time=   0.0s
[CV 5/5] END C=0.1, degree=2, gamma=0.05, kernel=poly;, score=0.574 total time=   0.0s
[CV 1/5] END C=0.1, degree=3, gamma=0, kernel=poly;, score=0.565 total time=   0.0s
[CV 2/5] END C=0.1, degree=3, gamma=0, kernel=poly;, score=0.565 total time=   0.0s
[CV 3/5] END C=0.1, degree=3, gamma=0, kernel=pol

[CV 1/5] END C=1, degree=3, gamma=0.01, kernel=poly;, score=0.565 total time=   0.0s
[CV 2/5] END C=1, degree=3, gamma=0.01, kernel=poly;, score=0.565 total time=   0.0s
[CV 3/5] END C=1, degree=3, gamma=0.01, kernel=poly;, score=0.580 total time=   0.0s
[CV 4/5] END C=1, degree=3, gamma=0.01, kernel=poly;, score=0.574 total time=   0.0s
[CV 5/5] END C=1, degree=3, gamma=0.01, kernel=poly;, score=0.574 total time=   0.0s
[CV 1/5] END C=1, degree=3, gamma=0.02, kernel=poly;, score=0.565 total time=   0.0s
[CV 2/5] END C=1, degree=3, gamma=0.02, kernel=poly;, score=0.565 total time=   0.0s
[CV 3/5] END C=1, degree=3, gamma=0.02, kernel=poly;, score=0.580 total time=   0.0s
[CV 4/5] END C=1, degree=3, gamma=0.02, kernel=poly;, score=0.574 total time=   0.0s
[CV 5/5] END C=1, degree=3, gamma=0.02, kernel=poly;, score=0.574 total time=   0.0s
[CV 1/5] END C=1, degree=3, gamma=0.03, kernel=poly;, score=0.565 total time=   0.0s
[CV 2/5] END C=1, degree=3, gamma=0.03, kernel=poly;, score=0.565

[CV 1/5] END C=10, degree=3, gamma=0.04, kernel=poly;, score=0.652 total time=   0.0s
[CV 2/5] END C=10, degree=3, gamma=0.04, kernel=poly;, score=0.667 total time=   0.0s
[CV 3/5] END C=10, degree=3, gamma=0.04, kernel=poly;, score=0.681 total time=   0.0s
[CV 4/5] END C=10, degree=3, gamma=0.04, kernel=poly;, score=0.750 total time=   0.0s
[CV 5/5] END C=10, degree=3, gamma=0.04, kernel=poly;, score=0.691 total time=   0.0s
[CV 1/5] END C=10, degree=3, gamma=0.05, kernel=poly;, score=0.681 total time=   0.0s
[CV 2/5] END C=10, degree=3, gamma=0.05, kernel=poly;, score=0.667 total time=   0.0s
[CV 3/5] END C=10, degree=3, gamma=0.05, kernel=poly;, score=0.681 total time=   0.0s
[CV 4/5] END C=10, degree=3, gamma=0.05, kernel=poly;, score=0.750 total time=   0.0s
[CV 5/5] END C=10, degree=3, gamma=0.05, kernel=poly;, score=0.750 total time=   0.0s
[CV 1/5] END C=10, degree=4, gamma=0, kernel=poly;, score=0.565 total time=   0.0s
[CV 2/5] END C=10, degree=4, gamma=0, kernel=poly;, score

GridSearchCV(cv=5, estimator=SVC(),
             param_grid=[{'C': [0.1, 1, 10], 'kernel': ['linear']},
                         {'C': [0.1, 1, 10],
                          'gamma': [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8,
                                    0.9],
                          'kernel': ['rbf']},
                         {'C': [0.1, 1, 10], 'degree': [0, 2, 3, 4],
                          'gamma': [0, 0.01, 0.02, 0.03, 0.04, 0.05],
                          'kernel': ['poly']},
                         {'C': [0.1, 1, 10], 'kernel': ['sigmoid']}],
             scoring='accuracy', verbose=3)

In [110]:
# examine the best model


# best score achieved during the GridSearchCV (Takes the mean of all the CV folds) 
print('GridSearch CV best score : {:.4f}\n\n'.format(svc_grid_search.best_score_))


# print parameters that give the best results
print('Parameters that give the best results :','\n\n', (svc_grid_search.best_params_))


# print estimator that was chosen by the GridSearch
print('\n\nEstimator that was chosen by the search :','\n\n', (svc_grid_search.best_estimator_))

GridSearch CV best score : 0.7145


Parameters that give the best results : 

 {'C': 1, 'kernel': 'linear'}


Estimator that was chosen by the search : 

 SVC(C=1, kernel='linear')


In [111]:
y_pred_svc=svc_grid_search.predict(X_test)

In [112]:
print(classification_report(y_test,y_pred_svc)) 

              precision    recall  f1-score   support

           0       0.88      0.62      0.73        45
           1       0.69      0.90      0.78        41

    accuracy                           0.76        86
   macro avg       0.78      0.76      0.75        86
weighted avg       0.78      0.76      0.75        86



In [113]:
train_acc_svm=svc_grid_search.best_score_
acc_svm=classification_report(y_test,y_pred_svc,output_dict=True)['accuracy']

In [114]:
#Xgboost - Using gridserch Cross validation for optimal training through a grid of parameters

from xgboost import XGBClassifier
estimator = XGBClassifier(
    objective= 'binary:logistic',
    nthread=4,
    seed=42
)

xgb_parameters = {
    'max_depth': range (2, 10, 1),
    'n_estimators': range(60, 220, 40),
    'learning_rate': [0.1, 0.01, 0.05]
}


xgb_grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=xgb_parameters,
    scoring = 'accuracy',
    cv = 5,
    verbose=3
)



xgb_grid_search.fit(X_train, y_train)


Fitting 5 folds for each of 96 candidates, totalling 480 fits
[CV 1/5] END learning_rate=0.1, max_depth=2, n_estimators=60;, score=0.739 total time=   0.0s
[CV 2/5] END learning_rate=0.1, max_depth=2, n_estimators=60;, score=0.783 total time=   0.0s
[CV 3/5] END learning_rate=0.1, max_depth=2, n_estimators=60;, score=0.855 total time=   0.0s
[CV 4/5] END learning_rate=0.1, max_depth=2, n_estimators=60;, score=0.868 total time=   0.0s
[CV 5/5] END learning_rate=0.1, max_depth=2, n_estimators=60;, score=0.853 total time=   0.0s
[CV 1/5] END learning_rate=0.1, max_depth=2, n_estimators=100;, score=0.768 total time=   0.0s
[CV 2/5] END learning_rate=0.1, max_depth=2, n_estimators=100;, score=0.768 total time=   0.0s
[CV 3/5] END learning_rate=0.1, max_depth=2, n_estimators=100;, score=0.855 total time=   0.0s
[CV 4/5] END learning_rate=0.1, max_depth=2, n_estimators=100;, score=0.882 total time=   0.0s
[CV 5/5] END learning_rate=0.1, max_depth=2, n_estimators=100;, score=0.838 total time= 

[CV 3/5] END learning_rate=0.1, max_depth=6, n_estimators=100;, score=0.768 total time=   0.1s
[CV 4/5] END learning_rate=0.1, max_depth=6, n_estimators=100;, score=0.794 total time=   0.1s
[CV 5/5] END learning_rate=0.1, max_depth=6, n_estimators=100;, score=0.809 total time=   0.1s
[CV 1/5] END learning_rate=0.1, max_depth=6, n_estimators=140;, score=0.739 total time=   0.1s
[CV 2/5] END learning_rate=0.1, max_depth=6, n_estimators=140;, score=0.739 total time=   0.1s
[CV 3/5] END learning_rate=0.1, max_depth=6, n_estimators=140;, score=0.768 total time=   0.1s
[CV 4/5] END learning_rate=0.1, max_depth=6, n_estimators=140;, score=0.779 total time=   0.1s
[CV 5/5] END learning_rate=0.1, max_depth=6, n_estimators=140;, score=0.794 total time=   0.1s
[CV 1/5] END learning_rate=0.1, max_depth=6, n_estimators=180;, score=0.739 total time=   0.1s
[CV 2/5] END learning_rate=0.1, max_depth=6, n_estimators=180;, score=0.754 total time=   0.1s
[CV 3/5] END learning_rate=0.1, max_depth=6, n_est

[CV 5/5] END learning_rate=0.01, max_depth=2, n_estimators=140;, score=0.735 total time=   0.0s
[CV 1/5] END learning_rate=0.01, max_depth=2, n_estimators=180;, score=0.710 total time=   0.0s
[CV 2/5] END learning_rate=0.01, max_depth=2, n_estimators=180;, score=0.754 total time=   0.1s
[CV 3/5] END learning_rate=0.01, max_depth=2, n_estimators=180;, score=0.797 total time=   0.1s
[CV 4/5] END learning_rate=0.01, max_depth=2, n_estimators=180;, score=0.868 total time=   0.1s
[CV 5/5] END learning_rate=0.01, max_depth=2, n_estimators=180;, score=0.765 total time=   0.1s
[CV 1/5] END learning_rate=0.01, max_depth=3, n_estimators=60;, score=0.725 total time=   0.0s
[CV 2/5] END learning_rate=0.01, max_depth=3, n_estimators=60;, score=0.754 total time=   0.0s
[CV 3/5] END learning_rate=0.01, max_depth=3, n_estimators=60;, score=0.797 total time=   0.0s
[CV 4/5] END learning_rate=0.01, max_depth=3, n_estimators=60;, score=0.853 total time=   0.0s
[CV 5/5] END learning_rate=0.01, max_depth=3

[CV 3/5] END learning_rate=0.01, max_depth=7, n_estimators=60;, score=0.768 total time=   0.0s
[CV 4/5] END learning_rate=0.01, max_depth=7, n_estimators=60;, score=0.765 total time=   0.0s
[CV 5/5] END learning_rate=0.01, max_depth=7, n_estimators=60;, score=0.765 total time=   0.0s
[CV 1/5] END learning_rate=0.01, max_depth=7, n_estimators=100;, score=0.768 total time=   0.1s
[CV 2/5] END learning_rate=0.01, max_depth=7, n_estimators=100;, score=0.652 total time=   0.1s
[CV 3/5] END learning_rate=0.01, max_depth=7, n_estimators=100;, score=0.754 total time=   0.1s
[CV 4/5] END learning_rate=0.01, max_depth=7, n_estimators=100;, score=0.779 total time=   0.1s
[CV 5/5] END learning_rate=0.01, max_depth=7, n_estimators=100;, score=0.765 total time=   0.1s
[CV 1/5] END learning_rate=0.01, max_depth=7, n_estimators=140;, score=0.768 total time=   0.1s
[CV 2/5] END learning_rate=0.01, max_depth=7, n_estimators=140;, score=0.681 total time=   0.1s
[CV 3/5] END learning_rate=0.01, max_depth=

[CV 3/5] END learning_rate=0.05, max_depth=3, n_estimators=140;, score=0.841 total time=   0.0s
[CV 4/5] END learning_rate=0.05, max_depth=3, n_estimators=140;, score=0.868 total time=   0.0s
[CV 5/5] END learning_rate=0.05, max_depth=3, n_estimators=140;, score=0.824 total time=   0.0s
[CV 1/5] END learning_rate=0.05, max_depth=3, n_estimators=180;, score=0.768 total time=   0.1s
[CV 2/5] END learning_rate=0.05, max_depth=3, n_estimators=180;, score=0.768 total time=   0.1s
[CV 3/5] END learning_rate=0.05, max_depth=3, n_estimators=180;, score=0.841 total time=   0.1s
[CV 4/5] END learning_rate=0.05, max_depth=3, n_estimators=180;, score=0.868 total time=   0.1s
[CV 5/5] END learning_rate=0.05, max_depth=3, n_estimators=180;, score=0.838 total time=   0.1s
[CV 1/5] END learning_rate=0.05, max_depth=4, n_estimators=60;, score=0.739 total time=   0.0s
[CV 2/5] END learning_rate=0.05, max_depth=4, n_estimators=60;, score=0.783 total time=   0.0s
[CV 3/5] END learning_rate=0.05, max_depth

[CV 5/5] END learning_rate=0.05, max_depth=7, n_estimators=180;, score=0.794 total time=   0.2s
[CV 1/5] END learning_rate=0.05, max_depth=8, n_estimators=60;, score=0.754 total time=   0.0s
[CV 2/5] END learning_rate=0.05, max_depth=8, n_estimators=60;, score=0.710 total time=   0.0s
[CV 3/5] END learning_rate=0.05, max_depth=8, n_estimators=60;, score=0.739 total time=   0.0s
[CV 4/5] END learning_rate=0.05, max_depth=8, n_estimators=60;, score=0.779 total time=   0.0s
[CV 5/5] END learning_rate=0.05, max_depth=8, n_estimators=60;, score=0.765 total time=   0.1s
[CV 1/5] END learning_rate=0.05, max_depth=8, n_estimators=100;, score=0.710 total time=   0.1s
[CV 2/5] END learning_rate=0.05, max_depth=8, n_estimators=100;, score=0.725 total time=   0.1s
[CV 3/5] END learning_rate=0.05, max_depth=8, n_estimators=100;, score=0.783 total time=   0.1s
[CV 4/5] END learning_rate=0.05, max_depth=8, n_estimators=100;, score=0.765 total time=   0.1s
[CV 5/5] END learning_rate=0.05, max_depth=8,

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     gamma=None, gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_bin=None,
                                     max_ca...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                  

In [116]:
# examine the best model


# best score achieved during the GridSearchCV (Takes the mean of all the CV folds) 
print('GridSearch CV best score : {:.4f}\n\n'.format(xgb_grid_search.best_score_))


# print parameters that give the best results
print('Parameters that give the best results :','\n\n', (xgb_grid_search.best_params_))


# print estimator that was chosen by the GridSearch
print('\n\nEstimator that was chosen by the search :','\n\n', (xgb_grid_search.best_estimator_))

GridSearch CV best score : 0.8224


Parameters that give the best results : 

 {'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 100}


Estimator that was chosen by the search : 

 XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.1, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=2, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=4, nthread=4, num_parallel_tree=1, predictor='auto',
              random_state=42, reg_alpha=0, ...)


In [117]:
y_pred_xgb=xgb_grid_search.predict(X_test)

In [118]:
print(classification_report(y_test,y_pred_xgb))

              precision    recall  f1-score   support

           0       0.90      0.58      0.70        45
           1       0.67      0.93      0.78        41

    accuracy                           0.74        86
   macro avg       0.78      0.75      0.74        86
weighted avg       0.79      0.74      0.74        86



In [119]:
hd = list(X_train.columns)
for i, f in zip(hd, xgb_grid_search.best_estimator_.feature_importances_):
     print(i,round(f*100,2))

Age 5.2
Time_at_address 3.4
Time_employed 18.91
Time_bank 19.49
Home_Expn 13.9
Balance 7.44
Res_status_owner 1.43
Res_status_rent 0.0
Liab_ref_f 4.98
Liab_ref_t 0.0
Job_status_governmen 0.0
Job_status_military 0.0
Job_status_private_s 0.0
Job_status_retired 0.0
Job_status_self_empl 0.0
Job_status_student 0.0
Job_status_unemploye 7.39
Acc_ref_given 0.0
Acc_ref_oth_inst_ 0.0
Telephone_given 0.0
Telephone_not_given 0.0
Sex_F 0.0
Sex_M 0.0
Occupation_creative_ 2.58
Occupation_driver 0.0
Occupation_executive 0.0
Occupation_guard_etc 0.0
Occupation_labourer 0.0
Occupation_manager 0.0
Occupation_office_st 3.66
Occupation_productio 0.0
Occupation_professio 5.27
Occupation_sales 0.0
Occupation_semi_pro 6.36
Occupation_unemploye 0.0


In [120]:
train_acc_xgb=xgb_grid_search.best_score_
acc_xgb=classification_report(y_test,y_pred_xgb,output_dict=True)['accuracy']

In [121]:
results = pd.DataFrame({
    'Model': ['Decision Tree', 'SVM', 
              'XGBoost'],
    'Train Score': [train_acc_dt,train_acc_svm,train_acc_xgb],
    'Test Score': [acc_dt, acc_svm, acc_xgb]})
result_df = results.sort_values(by='Test Score', ascending=False)
result_df = result_df.set_index('Model')
result_df


,Train Score,Test Score
Model,,
SVM,0.714450,0.755814
XGBoost,0.822379,0.744186
Decision Tree,0.702771,0.697674


#### Inference: 

- Above results help us draw a comparison between performance of Decision Trees, SVM and XGboost
- We get highest score in training for Xgboost, however the score on test data is similar for SVM and Xgboost
- We also get the list of top variables for Decision trees and Xgboost, and they look mostly similar with time_bank and time_employed coming as the top variables

### 2) Apply XGBoost regression algorithm on the Boston housing data set and compare its performance against Linear regression.

In [123]:
bh_df= pd.read_excel('boston housing.xls')
bh_df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PT,B,LSTAT,MV
0,0.00632,18.0,2.31,0,0.538,6.575,65.199997,4.0900,1,296,15.300000,396.899994,4.98,24.000000
1,0.02731,0.0,7.07,0,0.469,6.421,78.900002,4.9671,2,242,17.799999,396.899994,9.14,21.600000
2,0.02729,0.0,7.07,0,0.469,7.185,61.099998,4.9671,2,242,17.799999,392.829987,4.03,34.700001
3,0.03237,0.0,2.18,0,0.458,6.998,45.799999,6.0622,3,222,18.700001,394.630005,2.94,33.400002
4,0.06905,0.0,2.18,0,0.458,7.147,54.200001,6.0622,3,222,18.700001,396.899994,5.33,36.200001


In [124]:
bh_df.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PT,B,LSTAT,MV
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674030,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148862,2.105710,8.707259,168.537116,2.164946,91.294863,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377487,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440002,11.360000,21.200001
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.074999,5.188425,24.000000,666.000000,20.200001,396.225006,16.954999,25.000000
max,88.976196,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.899994,37.970001,50.000000


In [125]:
bh_df.dtypes

CRIM     float64
ZN       float64
INDUS    float64
CHAS       int64
NOX      float64
RM       float64
AGE      float64
DIS      float64
RAD        int64
TAX        int64
PT       float64
B        float64
LSTAT    float64
MV       float64
dtype: object

In [141]:
#splitting data into train-test groups

X=bh_df.drop('MV',axis=1)
y=bh_df['MV']

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2, random_state= 1)

In [142]:
print(X.shape,X_train.shape,X_test.shape)

(506, 13) (404, 13) (102, 13)


In [143]:
from sklearn.linear_model import LinearRegression

lm=LinearRegression()

lm.fit(X_train,y_train)

LinearRegression()

In [144]:
y_pred_lm=lm.predict(X_test)

In [151]:
#Calculating Root mean squared Error (RMSE)
from sklearn.metrics import mean_squared_error

rms_dt = mean_squared_error(y_test, y_pred_lm, squared=False)
rms_dt

4.83537351816394

In [147]:
#Xgboost - Using gridserch Cross validation for optimal training through a grid of parameters
#neg_mean_squared_error used as grid search tries to maximize the performance metrics
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
estimator = XGBRegressor(
    objective= 'reg:squarederror',
    nthread=4,
    seed=42
)

xgb_parameters = {
    'max_depth': range (2, 10, 1),
    'n_estimators': range(60, 220, 40),
    'learning_rate': [0.1, 0.01, 0.05]
}


xgb_grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=xgb_parameters,
    scoring = 'neg_mean_squared_error',
    cv = 5,
    verbose=3
)



xgb_grid_search.fit(X_train, y_train)


Fitting 5 folds for each of 96 candidates, totalling 480 fits
[CV 1/5] END learning_rate=0.1, max_depth=2, n_estimators=60;, score=-10.264 total time=   0.0s
[CV 2/5] END learning_rate=0.1, max_depth=2, n_estimators=60;, score=-13.762 total time=   0.0s
[CV 3/5] END learning_rate=0.1, max_depth=2, n_estimators=60;, score=-19.929 total time=   0.0s
[CV 4/5] END learning_rate=0.1, max_depth=2, n_estimators=60;, score=-24.219 total time=   0.0s
[CV 5/5] END learning_rate=0.1, max_depth=2, n_estimators=60;, score=-14.564 total time=   0.0s
[CV 1/5] END learning_rate=0.1, max_depth=2, n_estimators=100;, score=-9.758 total time=   0.0s
[CV 2/5] END learning_rate=0.1, max_depth=2, n_estimators=100;, score=-10.887 total time=   0.0s
[CV 3/5] END learning_rate=0.1, max_depth=2, n_estimators=100;, score=-20.040 total time=   0.0s
[CV 4/5] END learning_rate=0.1, max_depth=2, n_estimators=100;, score=-22.984 total time=   0.0s
[CV 5/5] END learning_rate=0.1, max_depth=2, n_estimators=100;, score=-

[CV 3/5] END learning_rate=0.1, max_depth=6, n_estimators=100;, score=-10.057 total time=   0.1s
[CV 4/5] END learning_rate=0.1, max_depth=6, n_estimators=100;, score=-22.406 total time=   0.1s
[CV 5/5] END learning_rate=0.1, max_depth=6, n_estimators=100;, score=-10.458 total time=   0.1s
[CV 1/5] END learning_rate=0.1, max_depth=6, n_estimators=140;, score=-8.963 total time=   0.1s
[CV 2/5] END learning_rate=0.1, max_depth=6, n_estimators=140;, score=-10.397 total time=   0.1s
[CV 3/5] END learning_rate=0.1, max_depth=6, n_estimators=140;, score=-9.957 total time=   0.1s
[CV 4/5] END learning_rate=0.1, max_depth=6, n_estimators=140;, score=-22.323 total time=   0.1s
[CV 5/5] END learning_rate=0.1, max_depth=6, n_estimators=140;, score=-10.320 total time=   0.1s
[CV 1/5] END learning_rate=0.1, max_depth=6, n_estimators=180;, score=-9.010 total time=   0.1s
[CV 2/5] END learning_rate=0.1, max_depth=6, n_estimators=180;, score=-10.417 total time=   0.1s
[CV 3/5] END learning_rate=0.1, m

[CV 2/5] END learning_rate=0.01, max_depth=2, n_estimators=180;, score=-39.324 total time=   0.0s
[CV 3/5] END learning_rate=0.01, max_depth=2, n_estimators=180;, score=-27.508 total time=   0.0s
[CV 4/5] END learning_rate=0.01, max_depth=2, n_estimators=180;, score=-55.234 total time=   0.0s
[CV 5/5] END learning_rate=0.01, max_depth=2, n_estimators=180;, score=-34.758 total time=   0.0s
[CV 1/5] END learning_rate=0.01, max_depth=3, n_estimators=60;, score=-197.006 total time=   0.0s
[CV 2/5] END learning_rate=0.01, max_depth=3, n_estimators=60;, score=-190.996 total time=   0.0s
[CV 3/5] END learning_rate=0.01, max_depth=3, n_estimators=60;, score=-157.819 total time=   0.0s
[CV 4/5] END learning_rate=0.01, max_depth=3, n_estimators=60;, score=-232.240 total time=   0.0s
[CV 5/5] END learning_rate=0.01, max_depth=3, n_estimators=60;, score=-170.746 total time=   0.0s
[CV 1/5] END learning_rate=0.01, max_depth=3, n_estimators=100;, score=-98.869 total time=   0.0s
[CV 2/5] END learnin

[CV 2/5] END learning_rate=0.01, max_depth=7, n_estimators=60;, score=-189.816 total time=   0.0s
[CV 3/5] END learning_rate=0.01, max_depth=7, n_estimators=60;, score=-156.580 total time=   0.0s
[CV 4/5] END learning_rate=0.01, max_depth=7, n_estimators=60;, score=-231.888 total time=   0.0s
[CV 5/5] END learning_rate=0.01, max_depth=7, n_estimators=60;, score=-169.733 total time=   0.0s
[CV 1/5] END learning_rate=0.01, max_depth=7, n_estimators=100;, score=-97.454 total time=   0.1s
[CV 2/5] END learning_rate=0.01, max_depth=7, n_estimators=100;, score=-94.018 total time=   0.0s
[CV 3/5] END learning_rate=0.01, max_depth=7, n_estimators=100;, score=-76.813 total time=   0.1s
[CV 4/5] END learning_rate=0.01, max_depth=7, n_estimators=100;, score=-127.520 total time=   0.1s
[CV 5/5] END learning_rate=0.01, max_depth=7, n_estimators=100;, score=-84.461 total time=   0.1s
[CV 1/5] END learning_rate=0.01, max_depth=7, n_estimators=140;, score=-51.673 total time=   0.1s
[CV 2/5] END learni

[CV 3/5] END learning_rate=0.05, max_depth=3, n_estimators=100;, score=-11.158 total time=   0.0s
[CV 4/5] END learning_rate=0.05, max_depth=3, n_estimators=100;, score=-25.133 total time=   0.0s
[CV 5/5] END learning_rate=0.05, max_depth=3, n_estimators=100;, score=-11.125 total time=   0.0s
[CV 1/5] END learning_rate=0.05, max_depth=3, n_estimators=140;, score=-8.958 total time=   0.1s
[CV 2/5] END learning_rate=0.05, max_depth=3, n_estimators=140;, score=-11.967 total time=   0.1s
[CV 3/5] END learning_rate=0.05, max_depth=3, n_estimators=140;, score=-10.689 total time=   0.1s
[CV 4/5] END learning_rate=0.05, max_depth=3, n_estimators=140;, score=-24.115 total time=   0.0s
[CV 5/5] END learning_rate=0.05, max_depth=3, n_estimators=140;, score=-10.739 total time=   0.0s
[CV 1/5] END learning_rate=0.05, max_depth=3, n_estimators=180;, score=-9.074 total time=   0.1s
[CV 2/5] END learning_rate=0.05, max_depth=3, n_estimators=180;, score=-11.583 total time=   0.1s
[CV 3/5] END learning_

[CV 4/5] END learning_rate=0.05, max_depth=7, n_estimators=140;, score=-24.235 total time=   0.1s
[CV 5/5] END learning_rate=0.05, max_depth=7, n_estimators=140;, score=-10.766 total time=   0.1s
[CV 1/5] END learning_rate=0.05, max_depth=7, n_estimators=180;, score=-8.030 total time=   0.1s
[CV 2/5] END learning_rate=0.05, max_depth=7, n_estimators=180;, score=-10.580 total time=   0.1s
[CV 3/5] END learning_rate=0.05, max_depth=7, n_estimators=180;, score=-11.450 total time=   0.1s
[CV 4/5] END learning_rate=0.05, max_depth=7, n_estimators=180;, score=-24.035 total time=   0.1s
[CV 5/5] END learning_rate=0.05, max_depth=7, n_estimators=180;, score=-10.576 total time=   0.1s
[CV 1/5] END learning_rate=0.05, max_depth=8, n_estimators=60;, score=-10.728 total time=   0.0s
[CV 2/5] END learning_rate=0.05, max_depth=8, n_estimators=60;, score=-15.554 total time=   0.0s
[CV 3/5] END learning_rate=0.05, max_depth=8, n_estimators=60;, score=-11.714 total time=   0.0s
[CV 4/5] END learning_ra

GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    gamma=None, gpu_id=None, grow_policy=None,
                                    importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, max_bin=None,
                                    max_cat...
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None, n_estimators=100,
                                  

In [148]:
# examine the best model


# best score achieved during the GridSearchCV (Takes the mean of all the CV folds) - Model w/ Linear kernel comes out on top
print('GridSearch CV best score : {:.4f}\n\n'.format(xgb_grid_search.best_score_))


# print parameters that give the be"st results
print('Parameters that give the best results :','\n\n', (xgb_grid_search.best_params_))


# print estimator that was chosen by the GridSearch
print('\n\nEstimator that was chosen by the search :','\n\n', (xgb_grid_search.best_estimator_))

GridSearch CV best score : -12.3715


Parameters that give the best results : 

 {'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 180}


Estimator that was chosen by the search : 

 XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.1, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=180, n_jobs=4,
             nthread=4, num_parallel_tree=1, predictor='auto', random_state=42,
             reg_alpha=0, ...)


In [149]:
y_pred_xgb=xgb_grid_search.predict(X_test)

In [150]:
from sklearn.metrics import mean_squared_error

rms_xgb = mean_squared_error(y_test, y_pred_xgb, squared=False)
rms_xgb

3.045729486244794

In [152]:
results = pd.DataFrame({
    'Model': ['Linear Regression', 
              'XGBoost'],
    'RMSE': [rms_dt,rms_xgb]})
result_df = results.sort_values(by='RMSE', ascending=False)
result_df = result_df.set_index('Model')
result_df


,RMSE
Model,
Linear Regression,4.835374
XGBoost,3.045729


#### Inference: 

- Above results help us draw a comparison between performance of Linear regression and XGboost
- By using Xgboost, we are able to significantly reduce the error in predictions with RMSE of 3.04 as compared to RMSE of 4.83 with linear regression